# Notebook Configuration

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import norm
from surprise import Dataset, Reader, KNNBasic, accuracy
from sklearn.metrics.pairwise import cosine_similarity

# Pregunta 1

La metrícas de similitud del KNN se basan en encontrar aquellas más proximas a una (vecinos más cercanos de esta misma). En base a una distancia seleccionada, euclidiana, manhattahn entre otros y los K vecinos mas cercanos, selecciona aquellos items que colinden entre esos mismos.

* Ventajas:
** Para preferencias de items, siendo menor que usarios, brinda recomendaciones debido a que el enfoque es mas escalable y el computo se reduce, en un dataset de peliculas, siendo estas menores que usuarios, y es enriquecedor sacar las comparaciones entre estas mismas porque cada modelo debe ser una experiencia unica como hace netflix.

** Encuentra mejores patrones cuando la data esta por segmentos como se muestran en las peliculas, por categorias, en estos segementos mas granulares, el KNN reduce su computo y performa mejor en base a item-item.


* Limitaciones:
** No es escalable debido a la cantidad de puntos computados con el objetivo, complejidad alta. En una plataforma de peliculas abundantes como netflix podría saturarse.
** Para items que no sean bastante vistos, una matriz de ceros grandes generaria distancias no sinceradas, por lo que las recomendaciones podrían equivocarse y brindar una mala experiencia, siendo esto algo penetrante para peliculas pues se invierte mucho tiempo por parte del usuario.

# Pregunta 2

Get data

In [4]:
# # Cargar los datasets de calificaciones y películas
ratings = pd.read_csv('datasets/ml-1m/ratings.dat', sep='::', header=None, engine='python',
                      names=['userId', 'movieId', 'rating', 'timestamp'], encoding='latin-1')
movies  = pd.read_csv('datasets/ml-1m/movies.dat', sep='::',  header=None, engine='python',
                      names=['movieId', 'title', 'genres'], encoding='latin-1')

Join data

In [5]:
# Unir los datasets en base a 'movieId' para agregar los títulos
user_item_rating = pd.merge(ratings, movies[['movieId', 'title']], on='movieId')

# Seleccionar solo las columnas necesarias
user_item_rating = user_item_rating[['userId', 'title', 'rating']]

# Ordenar los datos por 'userId'
user_item_rating.sort_values(by='userId', inplace=True)

# Mostrar las primeras filas para verificar
user_item_rating.head()

,userId,title,rating
0,1,One Flew Over the Cuckoo's Nest (1975),5
29,1,"Close Shave, A (1995)",3
30,1,Antz (1998),4
31,1,"Girl, Interrupted (1999)",4
32,1,Hercules (1997),4


Select two users that have at least 5 movies in common

In [6]:
# Establecer el mínimo de calificaciones por película
min_ratings_per_movie = 5

# Filtrar las películas con al menos min_ratings_per_movie calificaciones
movie_counts = user_item_rating['title'].value_counts()
popular_movies = movie_counts[movie_counts >= min_ratings_per_movie].index

# Filtrar el dataset original
filtered_data = user_item_rating[user_item_rating['title'].isin(popular_movies)]
filtered_data.head()

,userId,title,rating
0,1,One Flew Over the Cuckoo's Nest (1975),5
29,1,"Close Shave, A (1995)",3
30,1,Antz (1998),4
31,1,"Girl, Interrupted (1999)",4
32,1,Hercules (1997),4


In [7]:
# Contar cuántas películas ha calificado cada usuario
user_rating_counts_filtered = filtered_data['userId'].value_counts()

Generate user - item matrix

In [8]:
# Crear la matriz usuario-ítem
user_item_matrix = filtered_data.pivot_table(index='userId', columns='title', values='rating')

# Ordenar los IDs de usuario por cantidad de calificaciones
sorted_user_ids = user_rating_counts_filtered.index

# Ordenar la matriz según usuarios más activos
user_item_matrix_sorted = user_item_matrix.loc[sorted_user_ids]
user_item_matrix_sorted.head()

title,"$1,000,000 Duck (1971)",'Night Mother (1986),'Til There Was You (1997),"'burbs, The (1989)",...And Justice for All (1979),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),"13th Warrior, The (1999)",...,Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Young Sherlock Holmes (1985),Young and Innocent (1937),Your Friends and Neighbors (1998),"Zed & Two Noughts, A (1985)",Zero Effect (1998),Zeus and Roxanne (1997),eXistenZ (1999)
userId,,,,,,,,,,,,,,,,,,,,,
4169,NaN,5.0,NaN,NaN,3.0,NaN,4.0,4.0,5.0,4.0,...,3.0,3.0,NaN,3.0,3.0,3.0,3.0,3.0,NaN,4.0
1680,3.0,5.0,3.0,3.0,5.0,NaN,5.0,3.0,NaN,1.0,...,1.0,1.0,NaN,3.0,NaN,5.0,3.0,4.0,NaN,5.0
4277,NaN,NaN,NaN,NaN,5.0,NaN,4.0,NaN,5.0,4.0,...,4.0,3.0,2.0,4.0,NaN,NaN,3.0,5.0,NaN,4.0
1941,4.0,NaN,NaN,1.0,NaN,2.0,5.0,3.0,5.0,1.0,...,4.0,3.0,NaN,3.0,NaN,NaN,NaN,NaN,4.0,NaN
1181,NaN,NaN,NaN,NaN,2.0,2.0,3.0,3.0,2.0,4.0,...,2.0,2.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,4.0


Get users

In [9]:
userid_1 = 4169
userid_2 = 1680

# get values
values = user_item_matrix_sorted[user_item_matrix_sorted.index.isin([userid_1, userid_2])]

# calculate correlation
values.T.corr(method = 'pearson')

userId,4169,1680
userId,,
4169,1.000000,0.238442
1680,0.238442,1.000000


Interpretacion:

Se muestra una correlacion positiva entre los dos usuarios con respecto a sus puntuaciones, esto quiere decir que a medida que uno aumenta sus puntuaciones el otro tambien lo hace. Además, al tener valores en la misma tendencia, pueden tener un patron de consumo similar. Sin embargo, como el valor no es una correlacion fuerte, quiere decir que tambien para las mismas peliculas, han puntuado de manera diferente o alejada

# Pregunta 3

In [17]:

def cosine_dist(A, B):
  return np.dot(A,B)/(norm(A)*norm(B))

In [21]:
count_movies = filtered_data.title.value_counts()
filtered_count_movies = count_movies[(count_movies >= 50) & (count_movies <= 60)]

# get top 3
top3 = filtered_count_movies[:3]

# get names
top3_names = top3.index.tolist()

# select movies and their scores
top3_score = user_item_matrix_sorted[top3_names].fillna(0).copy()

# calculate sim
cosine_similarity_1_2 = cosine_dist(top3_score.iloc[:, 0], top3_score.iloc[:, 1])
cosine_similarity_1_3 = cosine_dist(top3_score.iloc[:, 0], top3_score.iloc[:, 2])

cosine_similarity_1_2, cosine_similarity_1_3

(0.027256567834108552, 0.041427613249958056)

Interpretacion:

Usaria los items de las pelicula 1 para recomendar las faltantes en las peliculas 3 debido a la mayor similitud. Esto en base a preferencias mas no en tamaños como rating debido a que esta metrica no ve eso. Por otro se podrían brindar recomendaciones entre items si esta medida fuera mas cercana a 1, pues se entendería que estan superpuestas por el angulo pequeño que las separa

# Pregunta 4

Split into subsets

In [22]:
def Random_Holdout(df, test_size=0.25):
    """Divide los datos asegurando que cada usuario tenga al menos una interacción en el conjunto de entrenamiento"""
    # Seleccionar al azar un porcentaje de interacciones por usuario para entrenamiento
    train_df = df.groupby('userId').apply(lambda x: x.sample(frac=1 - test_size, random_state=42)).reset_index(drop=True)
    # El resto va al conjunto de prueba
    test_df = pd.concat([df, train_df]).drop_duplicates(keep=False)

    return train_df, test_df

# Aplicar la función al dataset filtrado
train_df, test_df = Random_Holdout(filtered_data, test_size=0.3)

/tmp/ipykernel_19519/2001034640.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_df = df.groupby('userId').apply(lambda x: x.sample(frac=1 - test_size, random_state=42)).reset_index(drop=True)


In [24]:
from surprise import Dataset, Reader, KNNBasic, accuracy

# Definir el rango de calificaciones
reader = Reader(rating_scale=(1, 5))

# Cargar datos de entrenamiento
# First column has to be users when user_based is True
trainset = Dataset.load_from_df(train_df[['userId', 'title', 'rating']], reader).build_full_trainset()

# Crear conjunto de prueba
testset = list(test_df[['userId', 'title', 'rating']].itertuples(index=False, name=None))

It is used item-item in order to use previous results to support last answer

In [25]:
# Configurar opciones del modelo KNN
sim_options = {'name': 'msd',  # Mean Squared Difference (distancia euclidiana)
               'user_based': False} # Filtrado Usuario-Usuario

# Crear el modelo KNN
knn = KNNBasic(k = 30, sim_options = sim_options)

# Entrenar el modelo
knn.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


Make predictions

In [26]:
# Hacer predicciones sobre el conjunto de prueba
predictions = knn.test(testset)

# Calcular el RMSE
rmse = accuracy.rmse(predictions)
print(f"RMSE del modelo: {rmse:.4f}")

RMSE: 0.9233
RMSE del modelo: 0.9233


In [27]:
# Convertir predicciones a DataFrame
predictions_df = pd.DataFrame(predictions, columns=['userId', 'movieId', 'real_rating', 'predicted_rating', 'details'])

# Mostrar las predicciones principales para el usuario 324
user_324_predictions = predictions_df[predictions_df.userId == 324].sort_values(by='predicted_rating', ascending=False)
user_324_predictions.head(10)

,userId,movieId,real_rating,predicted_rating,details
14467,324,Aladdin (1992),4,3.790093,"{'actual_k': 30, 'was_impossible': False}"
14464,324,"Matrix, The (1999)",5,3.722134,"{'actual_k': 30, 'was_impossible': False}"
14470,324,Star Trek: The Wrath of Khan (1982),5,3.695273,"{'actual_k': 30, 'was_impossible': False}"
14474,324,Enemy of the State (1998),3,3.686965,"{'actual_k': 30, 'was_impossible': False}"
14466,324,"Princess Bride, The (1987)",4,3.682759,"{'actual_k': 30, 'was_impossible': False}"
14463,324,"Sound of Music, The (1965)",4,3.652547,"{'actual_k': 30, 'was_impossible': False}"
14475,324,"Green Mile, The (1999)",4,3.636017,"{'actual_k': 30, 'was_impossible': False}"
14458,324,Remember the Titans (2000),3,3.635736,"{'actual_k': 30, 'was_impossible': False}"
14468,324,Ghost (1990),4,3.625816,"{'actual_k': 30, 'was_impossible': False}"
14462,324,"Usual Suspects, The (1995)",5,3.620741,"{'actual_k': 30, 'was_impossible': False}"


Global hit rate

In [28]:
# Definir umbral de relevancia
relevant_threshold = 4.0

# Inicializar contadores
hits = 0
total_relevant = 0

# Iterar sobre todas las predicciones
for _, row in predictions_df.iterrows():
    real_rating = row['real_rating']
    predicted_rating = row['predicted_rating']

    # Considerar películas con calificación real ≥ umbral como relevantes
    if real_rating >= relevant_threshold:
        total_relevant += 1
        # Si la predicción también es ≥ umbral, es un acierto
        if predicted_rating >= relevant_threshold:
            hits += 1

# Calcular el Hit Rate Global
hit_rate_global = hits / total_relevant if total_relevant > 0 else 0
print(f"Hit Rate Global: {hit_rate_global:.2f}")

Hit Rate Global: 0.42


Git rate per user

In [29]:
# Inicializar lista para almacenar hit rates por usuario
hit_rates = []

# Agrupar predicciones por usuario
for user_id, group in predictions_df.groupby('userId'):
    hits = 0
    total_relevant = 0

    for _, row in group.iterrows():
        real_rating = row['real_rating']
        predicted_rating = row['predicted_rating']

        # Considerar películas relevantes
        if real_rating >= relevant_threshold:
            total_relevant += 1
            if predicted_rating >= relevant_threshold:
                hits += 1

    # Calcular Hit Rate para el usuario
    if total_relevant > 0:
        hit_rate_user = hits / total_relevant
        hit_rates.append(hit_rate_user)

# Calcular el Hit Rate promedio
hit_rate_avg_user = sum(hit_rates) / len(hit_rates) if len(hit_rates) > 0 else 0
print(f"Hit Rate promedio por usuario: {hit_rate_avg_user:.2f}")

Hit Rate promedio por usuario: 0.39


Comentarios:

* Teniendo un Hitrate global de 0.42 se consigue alcanzar recomendaciones aceptadas. No obstante, habiendo partido del hecho que es importante no brindar recomendaciones erroneas debido al tiempo invertido por una pelicula no provechosa, se podría esperar una fuga de clientes. Por lo que se debería emplear otra métricas que tengan que ver con la efectividad más cercana de la predicion como presicion por ejemplo, que se enfoca en reducir los falsos positivos y por ende recomendaciones de mejor c a l i d a d.